# Email Enhancer Agent

This is an agent that can compose professional emails with enhanced content and improved subject lines.

This agent is built with smolagents using OpenAI's o4-mini model.

It uses a 4-step workflow to research, improve, enhance, and compose emails.


## 1. Setup
Let's install the relevant packages.


In [ ]:
%pip install --upgrade --quiet smolagents
%pip install --upgrade --quiet duckduckgo_search
%pip install --upgrade --quiet opentelemetry-api


: 

Let's import the necessary packages.


In [ ]:
import os
import time
import uuid
from typing import Dict, List, TypedDict, Annotated
from smolagents import OpenAIServerModel, DuckDuckGoSearchTool
from opentelemetry import trace



## 2. Environment Variables

Let's set the environment variables.


In [ ]:
from helpers import _set_env

_set_env("OPENAI_API_KEY")

## 3. LLM and Tools

Let's initialize the LLM and tools that our agent will use.


In [ ]:
# Initialize tracer
tracer = trace.get_tracer(__name__)

# Initialize the model
llm = OpenAIServerModel(model_id="gpt-4o-mini")
search_tool = DuckDuckGoSearchTool()


## 4. Helper Functions

Let's define helper functions for email composition and formatting.


In [5]:
def get_email_style(email_type: str, recipient: str, signature_name: str) -> Dict[str, str]:
    """Get email style configuration based on type."""
    email_styles = {
        "formal": {
            "greeting": f"Dear {recipient},",
            "closing": f"Kind regards,\n{signature_name}",
            "tone": "formal and professional"
        },
        "informal": {
            "greeting": f"Hi {recipient},",
            "closing": f"Best,\n{signature_name}",
            "tone": "friendly and casual"
        },
        "small": {
            "greeting": f"Hello {recipient},",
            "closing": f"Kind regards,\n{signature_name}",
            "tone": "concise and direct"
        }
    }
    return email_styles.get(email_type.lower(), email_styles["formal"])

def format_final_email(subject: str, greeting: str, content: str, closing: str) -> str:
    """Format the final email with proper structure."""
    return f"""Subject: {subject}

{greeting}

{content}

{closing}
"""


## 5. Nodes

Let's define the nodes that our agent will use.


In [6]:
def research_context(state):
    """Node to research email context and best practices."""
    email_details = state["email_details"]
    
    print("🔍: Researching email best practices...")
    
    # Research email best practices for the specific type and topic
    research_query = f"{email_details['email_type']} email best practices {email_details['subject']} professional writing"
    research_results = search_tool(research_query)
    
    print(f"🔍: Research completed for {email_details['email_type']} email about '{email_details['subject']}'")
    
    return {"email_details": email_details, "research_data": research_results}

def improve_subject(state):
    """Node to improve the email subject line."""
    email_details = state["email_details"]
    research_data = state["research_data"]
    
    print("📝: Improving subject line...")
    
    # Create a prompt to improve the subject line
    subject_prompt = f"""
    Based on this research data: {research_data[:500]}...
    
    Improve this email subject line to be more professional and appropriate for a {email_details['email_type']} email:
    Original subject: "{email_details['subject']}"
    
    Make it:
    - Professional and clear
    - Appropriate for {email_details['email_type']} communication
    - Action-oriented if needed
    - Specific and informative
    
    Return only the improved subject line, nothing else.
    """
    
    # Use the model to improve the subject
    improved_subject = llm([{"role": "user", "content": subject_prompt}]).content.strip()
    
    print(f"📝: Subject improved: '{improved_subject}'")
    
    return {
        "email_details": email_details,
        "research_data": research_data,
        "improved_subject": improved_subject
    }

def enhance_content(state):
    """Node to enhance the email content."""
    email_details = state["email_details"]
    research_data = state["research_data"]
    improved_subject = state["improved_subject"]
    
    print("✨: Enhancing email content...")
    
    content_prompt = f"""
    Based on this research: {research_data[:500]}...
    
    Enhance this email content for a {email_details['email_type']} email:
    Original content: "{email_details['main_content']}"
    
    Make it:
    - More polished and professional
    - Better grammar and structure
    - Appropriate tone for {email_details['email_type']} communication
    - Clear and effective
    - Engaging and well-structured
    
    Return only the enhanced content, nothing else.
    """
    
    enhanced_content = llm([{"role": "user", "content": content_prompt}]).content.strip()
    
    print("✨: Content enhanced with better grammar and structure")
    
    return {
        "email_details": email_details,
        "research_data": research_data,
        "improved_subject": improved_subject,
        "enhanced_content": enhanced_content
    }

def compose_final_email(state):
    """Node to compose the final email."""
    email_details = state["email_details"]
    improved_subject = state["improved_subject"]
    enhanced_content = state["enhanced_content"]
    
    print("📧: Composing final email...")
    
    # Get email style
    style = get_email_style(
        email_details["email_type"],
        email_details["recipient"],
        email_details["signature_name"]
    )
    
    # Compose the final email
    final_email = format_final_email(
        improved_subject,
        style["greeting"],
        enhanced_content,
        style["closing"]
    )
    
    print(f"📧: Final email composed in {style['tone']} style")
    
    return {
        "email_details": email_details,
        "research_data": state["research_data"],
        "improved_subject": improved_subject,
        "enhanced_content": enhanced_content,
        "final_email": final_email
    }


## 6. Workflow Setup

Let's define the state and the workflow that our agent will use.

The workflow defines how our agent processes emails:

1. Research email best practices
2. Improve the subject line
3. Enhance the content
4. Compose the final email

This uses a simple sequential workflow to manage state transitions between different processing nodes.


In [7]:
class EmailState(TypedDict):
    email_details: Dict[str, str]
    research_data: str
    improved_subject: str
    enhanced_content: str
    final_email: str

def execute_email_workflow(email_details: Dict[str, str]) -> Dict[str, str]:
    """Execute the complete email writing workflow"""
    
    print("🚀 Starting Gmail Writing Workflow")
    print("=" * 50)
    
    # Initialize state
    state = {
        "email_details": email_details,
        "research_data": "",
        "improved_subject": "",
        "enhanced_content": "",
        "final_email": ""
    }
    
    # Execute workflow steps in sequence
    try:
        start_time = time.time()
        
        # Step 1: Research context
        state = research_context(state)
        
        # Step 2: Improve subject
        state = improve_subject(state)
        
        # Step 3: Enhance content
        state = enhance_content(state)
        
        # Step 4: Compose final email
        state = compose_final_email(state)
        
        end_time = time.time()
        execution_time = end_time - start_time
        
        print(f"\n🎉 Workflow completed successfully in {execution_time:.2f} seconds!")
        return state
        
    except Exception as e:
        print(f"❌ Workflow failed: {e}")
        return state


## 7. Run the agent

Let's now run the agent. The user has to input email details and the agent will provide a professionally composed email.


In [ ]:
print("📧: Hi there. I can compose professional Gmail emails with enhanced content and improved subject lines. Let's start!")

# Email type selection
email_types = {
    "1": "formal",
    "2": "informal", 
    "3": "small"
}

print("\n📝 Email Types:")
print("1. Formal - Professional business emails")
print("2. Informal - Casual friendly emails")
print("3. Small - Concise direct emails")

# Get user choice for email type
while True:
    choice = input("\n👤 Choose email type (1-3): ").strip()
    if choice in email_types:
        email_type = email_types[choice]
        print(f"✅ Selected: {email_type.title()} email")
        break
    else:
        print("❌ Invalid choice. Please enter 1, 2, or 3")

# Get email details
print("\n📋 Email Details:")
recipient = input("👤 Recipient name/email: ")
subject = input("📄 Email subject: ")
main_content = input("✍️ Main message content: ")
signature_name = input("✒️ Your name for signature: ")

# Create email details dictionary
email_details = {
    "email_type": email_type,
    "recipient": recipient,
    "subject": subject,
    "main_content": main_content,
    "signature_name": signature_name
}

# Execute workflow
result = execute_email_workflow(email_details)

print("\n📧:", result["final_email"])


📧: Hi there. I can compose professional Gmail emails with enhanced content and improved subject lines. Let's start!

📝 Email Types:
1. Formal - Professional business emails
2. Informal - Casual friendly emails
3. Small - Concise direct emails

## 8. Analytics and metrics

Let's now review the execution metrics for our agent run.


In [ ]:
# Simple analytics for the workflow
def analyze_workflow_performance(result: Dict[str, str]):
    """Analyze the workflow performance"""
    
    email_details = result["email_details"]
    
    print("📊 Workflow Analytics:")
    print("=" * 30)
    print(f"Email Type: {email_details['email_type']}")
    print(f"Original Subject Length: {len(email_details['subject'])} chars")
    print(f"Improved Subject Length: {len(result['improved_subject'])} chars")
    print(f"Original Content Length: {len(email_details['main_content'])} chars")
    print(f"Enhanced Content Length: {len(result['enhanced_content'])} chars")
    print(f"Research Data Length: {len(result['research_data'])} chars")
    
    # Improvement metrics
    subject_improvement = len(result['improved_subject']) - len(email_details['subject'])
    content_improvement = len(result['enhanced_content']) - len(email_details['main_content'])
    
    print(f"\n📈 Improvements:")
    print(f"Subject Enhancement: {'+' if subject_improvement > 0 else ''}{subject_improvement} chars")
    print(f"Content Enhancement: {'+' if content_improvement > 0 else ''}{content_improvement} chars")
    
    print(f"\n🎯 Quality Metrics:")
    print(f"Research Quality: {'High' if len(result['research_data']) > 1000 else 'Medium' if len(result['research_data']) > 500 else 'Low'}")
    print(f"Content Expansion: {content_improvement / len(email_details['main_content']) * 100:.1f}%")

# Run analytics on the result
analyze_workflow_performance(result)
